# Graph Coloring Problem

In [1064]:
# imports
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import time

In [1065]:
# function to read a graph from a file
def read_file(filename: str):
  with open(filename, 'r') as f:
    # for each line
    for line in f:
      tokens = line.split()
      if (tokens[0] == 'c'):
        continue
      elif (tokens[0] == 'p'):
        # n = int(tokens[2])
        # m = int(tokens[3])
        G = nx.Graph()
      elif (tokens[0] == 'e'):
        G.add_edge(int(tokens[1]), int(tokens[2]))

  return G

## Heuristics

In [1066]:
# graph coloring functions
def LF(G: nx.Graph):
  ans: dict = nx.greedy_color(G, strategy='largest_first')
  return max(ans.values()) + 1

def SL(G: nx.Graph):
  ans: dict = nx.greedy_color(G, strategy='smallest_last')
  return max(ans.values()) + 1

def LFI(G: nx.Graph):
  ans: dict = nx.greedy_color(G, strategy='largest_first', interchange=True)
  return max(ans.values()) + 1

def SLI(G: nx.Graph):
  ans: dict = nx.greedy_color(G, strategy='smallest_last', interchange=True)
  return max(ans.values()) + 1

def DSatur(G: nx.Graph):
  ans: dict = nx.coloring.greedy_color(G, strategy='DSATUR')
  return max(ans.values()) + 1

def RLF(G: nx.Graph):
  ans: dict = nx.coloring.greedy_color(G, strategy='independent_set')
  return max(ans.values()) + 1

In [1067]:
GA_CROSSOVER_PROB = 0.9
GA_MUTATION_PROB = 0.3
GA_GENERATION_COUNT = 500
GA_POPULATION_SIZE = 100

## GA

In [1068]:
# genetic algorithm
class GAIndividual:
  def __init__(self, number_of_colors: int):
    self.chromosome = dict()
    self.fitness: int = 1e9
    self.number_of_colors = number_of_colors

  def randomize(self, G: nx.Graph):
    for node in G.nodes:
      self.chromosome[node] = np.random.randint(0, self.number_of_colors)
    self.calculate_fitness(G)

  def calculate_fitness(self, G: nx.Graph):
    self.fitness = 0
    for u, v in G.edges:
      if self.chromosome[u] == self.chromosome[v]:
        self.fitness += 1

  def mutate(self):
    node = np.random.choice(list(self.chromosome.keys()))
    self.chromosome[node] = np.random.randint(0, self.number_of_colors)

class GAPopulation:
  def __init__(self, number_of_colors: int, population_size: int):
    self.individuals = [GAIndividual(number_of_colors) for _ in range(population_size)]
    self.number_of_colors = number_of_colors
    self.population_size = population_size

  def randomize(self, G: nx.Graph):
    for individual in self.individuals:
      individual.randomize(G)
      individual.calculate_fitness(G)

  def crossover(self, G: nx.Graph):
    for _ in range(0, self.population_size, 2):
      if np.random.rand() < GA_CROSSOVER_PROB:
        self._crossover(G)

  def _crossover(self, G: nx.Graph):
    parents = np.random.choice(self.individuals, 2, replace=False)
    child = GAIndividual(self.number_of_colors)
    for node in G.nodes:
      if np.random.rand() < 0.5:
        child.chromosome[node] = parents[0].chromosome[node]
      else:
        child.chromosome[node] = parents[1].chromosome[node]

  def mutate(self, G: nx.Graph):
    for individual in self.individuals:
      if np.random.rand() < GA_MUTATION_PROB:
        individual.mutate()
        individual.calculate_fitness(G)

  def select(self):
    # 3 way tournament selection
    selected = []
    while len(selected) < self.population_size:
      candidates = np.random.choice(self.individuals, 3, replace=False)
      selected.append(min(candidates, key=lambda x: x.fitness))

    self.individuals = selected

  def get_best(self):
    return min(self.individuals, key=lambda x: x.fitness)

In [1069]:
# Genetic Algorithm
def genetic_algorithm(G: nx.Graph, number_of_colors: int):
  population = GAPopulation(number_of_colors, GA_POPULATION_SIZE)
  population.randomize(G)
  iteration_count = 0

  while iteration_count < GA_GENERATION_COUNT and population.get_best().fitness > 0:
    population.crossover(G)
    population.mutate(G)
    population.select()
    if population.get_best().fitness == 0:
      population = GAPopulation(number_of_colors, GA_POPULATION_SIZE)
      population.randomize(G)
    iteration_count += 1

  return population.get_best().fitness

# main function
def GA(G: nx.Graph):
  number_of_colors = LF(G)
  while True:
    if genetic_algorithm(G, number_of_colors) == 0:
      number_of_colors -= 1
    else:
      return number_of_colors + 1

## Tabu

In [1070]:
TABU_LIST_SIZE = 40
TABU_ITERATIONS = 500

from collections import deque
from copy import deepcopy

In [1071]:
# Tabu search
def tabu_search(G: nx.Graph, number_of_colors: int):
  tabu_list: deque[tuple] = deque(maxlen=TABU_LIST_SIZE)
  best_solution = GAIndividual(number_of_colors)
  best_solution.randomize(G)
  best_solution.calculate_fitness(G)
  current_solution = best_solution
  iteration_count = 0

  while iteration_count < TABU_ITERATIONS and best_solution.fitness > 0:
    best_neighbor = None
    for _ in range(20):
      neighbor = deepcopy(current_solution)
      vertex = np.random.choice(list(G.nodes))
      coloring = np.random.randint(0, number_of_colors)
      neighbor.chromosome[vertex] = coloring
      neighbor.calculate_fitness(G)
      if neighbor.fitness < best_solution.fitness:
        best_neighbor = neighbor

    if best_neighbor is not None:
      current_solution = best_neighbor
      if current_solution.fitness < best_solution.fitness:
        best_solution = current_solution

    tabu_list.append(tuple([vertex, coloring]))
    iteration_count += 1

  return best_solution.fitness

In [1072]:
def Tabu(G: nx.Graph):
  number_of_colors = max(G.degree, key=lambda x: x[1])[1] + 1
  while True:
    fitness = tabu_search(G, number_of_colors)
    if fitness == 0:
      number_of_colors -= 1
    else:
      return number_of_colors + 1

## Benchmark

In [1073]:
graph = read_file('datasets/le450_25b.col')

In [1074]:
def benchmark_algorithm(algorithm, G):
  start = time.perf_counter_ns()
  result = algorithm(G)
  end = time.perf_counter_ns()
  time_taken_in_seconds = (end - start) / 1e9
  print(f'{algorithm.__name__}\t\t{result}\t\t{time_taken_in_seconds:.4f}s')

In [ ]:
# run the algorithms
print('Algorithm\tColors\t\tTime')
benchmark_algorithm(LF, graph)
benchmark_algorithm(SL, graph)
benchmark_algorithm(LFI, graph)
benchmark_algorithm(SLI, graph)
benchmark_algorithm(DSatur, graph)
benchmark_algorithm(RLF, graph)
benchmark_algorithm(GA, graph)
benchmark_algorithm(Tabu, graph)